#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to mattyb678@gmail.comand will expire on October 11, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-62134 - Server binary: /Users/mberteaux/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1446337316.log
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [3]:
products.head()

Columns:
	name	str
	review	str
	rating	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
[10 rows x 3 columns]

#Build the word count vector for each review

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+
|           word_count          |
+-------------------------------+
| {'and': 5, '6': 1, 'stink'... |
| {'and': 3, 'love': 1, 'it'... |
| {'and': 2, 'quilt': 1, 'it... |
| {'ingenious': 1, 'and': 3,... |
| {'and': 2, 'parents!!': 1,... |
| {'and': 2, 'this': 2, 'her... |
| {'shop': 1, 'noble': 1, 'i... |
| {'and': 2, 'all': 1, 'righ... |
| {'and': 1, 'help': 1, 'giv... |
| {'journal.': 1, 'nanny': 1... |
+-------------------------------+
[10 rows x 4 columns]

In [32]:
graphlab.canvas.set_target('ipynb')

In [5]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [12]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [13]:
len(giraffe_reviews)

785

In [16]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [ ]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [16]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [17]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [18]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'and': 3, 'love': 1, 'it'... |    0    |   0   |     0     |    0    |  1   |
| {'and': 2, 'quilt': 1, 'it... |    0    |   0   |     0     |    0    |  0   |
| {'ingenious': 1, 'and': 3,... |    0    |   0   |     0     |    0    |  2   |
| {'and': 2, 'parents!!': 1,... |    0    |   1   |     0     |    0    |  0   |
| {'and': 2, 'this': 2, 'her... |    0    |   1   |     0     |    0    |  0   |
| {'shop': 1, 'noble': 1, 'i... |    0    |   0   |     0     |    0    |  0   |
| {'and': 2, 'all': 1, 'righ... |    0    |   0   |     0     |    0    |  0   |
| {'and': 1, 'help': 1, 'giv... |    0    |   0   |     0     |    0    |  0   |
| {'journal.': 1, 'nanny': 1... |    0    |   0   |     0     |    0    |  0   |
| {'all': 1, 'forget': 1, 'j... |    0    |   0   |     0     |    0    |  2   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+
| horrible | bad | terrible | awful | wow | hate | sentiment |
+----------+-----+----------+-------+-----+------+-----------+
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
+----------+-----+----------+-------+-----+------+-----------+
[10 rows x 16 columns]

##Let's train the sentiment classifier

In [42]:
train_data,test_data = products.random_split(.8, seed=0)

In [29]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 1.041994     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 2.062075     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 2.487556     | 0.92

#Evaluate the sentiment model

In [49]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 |      1       |        0        |  1461 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [33]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [46]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [47]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | He likes chewing on all th... |  5.0   |
| Vulli Sophie the Giraffe T... | My son loves this toy and ... |  5.0   |
| Vulli Sophie the Giraffe T... | There really should be a l... |  1.0   |
| Vulli Sophie the Giraffe T... | All the moms in my moms' g... |  5.0   |
| Vulli Sophie the Giraffe T... | I was a little skeptical o... |  5.0   |
| Vulli Sophie the Giraffe T... | I have been reading about ... |  5.0   |
| Vulli Sophie the Giraffe T... | My neice loves her sophie ... |  5.0   |
| Vulli Sophie the Giraffe T... | What a friendly face!  And... |  5.0   |
| Vulli Sophie the Giraffe T... | We got this just for my so... |  5.0   |
| Vulli Sophie the Giraffe T... | My baby seems to like this... |  3.0   |
+-------------------------------+-------------------------------+--------+
+---------------------+
| predicted_sentiment |
+---------------------+
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
+---------------------+
[10 rows x 4 columns]

##Sort the reviews based on the predicted sentiment and explore

In [48]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [49]:
giraffe_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | He likes chewing on all th... |  5.0   |
| Vulli Sophie the Giraffe T... | My son loves this toy and ... |  5.0   |
| Vulli Sophie the Giraffe T... | There really should be a l... |  1.0   |
| Vulli Sophie the Giraffe T... | All the moms in my moms' g... |  5.0   |
| Vulli Sophie the Giraffe T... | I was a little skeptical o... |  5.0   |
| Vulli Sophie the Giraffe T... | I have been reading about ... |  5.0   |
| Vulli Sophie the Giraffe T... | My neice loves her sophie ... |  5.0   |
| Vulli Sophie the Giraffe T... | What a friendly face!  And... |  5.0   |
| Vulli Sophie the Giraffe T... | We got this just for my so... |  5.0   |
| Vulli Sophie the Giraffe T... | My baby seems to like this... |  3.0   |
+-------------------------------+-------------------------------+--------+
+---------------------+
| predicted_sentiment |
+---------------------+
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
|    0.674625848386   |
+---------------------+
[10 rows x 4 columns]

##Most positive reviews for the giraffe

In [50]:
giraffe_reviews[0]['review']

'He likes chewing on all the parts especially the head and the ears! It has helped when he has been cranky because of teething. Great purchase!'

In [51]:
giraffe_reviews[1]['review']

'My son loves this toy and fits great in the diaper bag. Also very easy to clean, just run right under the water. Seems to help him when he is getting a tooth.'

##Show most negative reviews for giraffe

In [52]:
giraffe_reviews[-1]['review']

'I know several baits that are enjoying the soft, flexible Sophie giraffe. No wonder she made it to the Top Baby Products!'

In [53]:
giraffe_reviews[-2]['review']

'My baby who is currently teething love his Sophie. I little bit expensive for a chew toy but it does its job.'

In [6]:
def selected_word_count(dict, word):
    if word in dict:
        return dict[word]
    else:
        return 0

In [7]:
selected_word_count({'awesome':3}, 'awesome')

3

In [60]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [6]:
products.head()

Columns:
	name	str
	review	str
	rating	float

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
[10 rows x 3 columns]

In [8]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [9]:
for word in selected_words:
    products[word] = products['word_count'].apply(lambda dict: selected_word_count(dict, word))

In [19]:
products.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'and': 3, 'love': 1, 'it'... |    0    |   0   |     0     |    0    |  1   |
| {'and': 2, 'quilt': 1, 'it... |    0    |   0   |     0     |    0    |  0   |
| {'ingenious': 1, 'and': 3,... |    0    |   0   |     0     |    0    |  2   |
| {'and': 2, 'parents!!': 1,... |    0    |   1   |     0     |    0    |  0   |
| {'and': 2, 'this': 2, 'her... |    0    |   1   |     0     |    0    |  0   |
| {'shop': 1, 'noble': 1, 'i... |    0    |   0   |     0     |    0    |  0   |
| {'and': 2, 'all': 1, 'righ... |    0    |   0   |     0     |    0    |  0   |
| {'and': 1, 'help': 1, 'giv... |    0    |   0   |     0     |    0    |  0   |
| {'journal.': 1, 'nanny': 1... |    0    |   0   |     0     |    0    |  0   |
| {'all': 1, 'forget': 1, 'j... |    0    |   0   |     0     |    0    |  2   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+
| horrible | bad | terrible | awful | wow | hate | sentiment |
+----------+-----+----------+-------+-----+------+-----------+
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
+----------+-----+----------+-------+-----+------+-----------+
[10 rows x 16 columns]

In [20]:
for word in selected_words:
    num = products[word].sum()
    print word + " - " , num 

awesome -  2002
great -  42420
fantastic -  873
amazing -  1305
love -  40277
horrible -  659
bad -  3197
terrible -  673
awful -  345
wow -  131
hate -  1057


In [45]:
train_data,test_data = products.random_split(.8, seed=0)

In [46]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.091787     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.149707     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.207255     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [47]:
selected_words_model['coefficients'].sort('value')

Columns:
	name	str
	index	str
	class	int
	value	float

Rows: 12

Data:
+-----------+-------+-------+------------------+
|    name   | index | class |      value       |
+-----------+-------+-------+------------------+
|  terrible |  None |   1   |  -2.09049998487  |
|  horrible |  None |   1   |  -1.99651800559  |
|   awful   |  None |   1   |  -1.76469955631  |
|    hate   |  None |   1   |  -1.40916406276  |
|    bad    |  None |   1   | -0.985827369929  |
|    wow    |  None |   1   | -0.0541450123333 |
|   great   |  None |   1   |  0.883937894898  |
| fantastic |  None |   1   |  0.891303090304  |
|  amazing  |  None |   1   |  0.892802422508  |
|  awesome  |  None |   1   |  1.05800888878   |
+-----------+-------+-------+------------------+
[12 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [48]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 |      1       |        0        |  130  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [35]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [36]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [37]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [38]:
diaper_champ_reviews.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
| Baby Trend Diaper Champ | I LOOOVE this diaper pail!... |  5.0   |
| Baby Trend Diaper Champ | We researched all of the d... |  4.0   |
| Baby Trend Diaper Champ | My baby is now 8 months an... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | Diaper Champ or Diaper Gen... |  5.0   |
| Baby Trend Diaper Champ | Wow!  This is fabulous.  I... |  5.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I am one of those super-cr... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'all': 1, 'less': 1, "fri... |    0    |   0   |     0     |    0    |  0   |
| {'just': 1, 'over': 1, 'rw... |    0    |   0   |     0     |    0    |  1   |
| {'all': 2, 'just': 4, "don... |    0    |   0   |     0     |    0    |  0   |
| {"don't": 1, 'able': 2, 'o... |    0    |   2   |     0     |    0    |  0   |
| {'just': 3, 'money': 1, 's... |    0    |   0   |     0     |    0    |  2   |
| {'son': 2, 'all': 1, 'bags... |    0    |   0   |     0     |    0    |  0   |
| {'and': 4, 'this': 3, 'sti... |    0    |   0   |     0     |    0    |  0   |
| {'lysol': 1, 'all': 2, 'ba... |    0    |   0   |     0     |    0    |  0   |
| {'just': 1, '-': 3, 'both'... |    0    |   0   |     0     |    0    |  0   |
| {'all': 1, 'humid': 1, 'ju... |    0    |   0   |     0     |    0    |  1   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+---------------------+
| horrible | bad | terrible | awful | wow | hate | sentiment | predicted_sentiment |
+----------+-----+----------+-------+-----+------+-----------+---------------------+
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999937267   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999917406   |
|    0     |  1  |    0     |   0   |  0  |  0   |     1     |    0.999999899509   |
|    0     |  1  |    0     |   0   |  0  |  0   |     1     |    0.999999836182   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999824745   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999759315   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999692111   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999642488   |
|    1     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999604504   |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |    0.999999486804   |
+----------+-----+----------+-------+-----+------+-----------+---------------------+
[10 rows x 17 columns]

In [39]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290673]

In [40]:
diaper_champ_reviews[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [50]:
test_data.head()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int

Rows: 10

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Nature's Lullabies First Y... | Space for monthly photos, ... |  5.0   |
| Nature's Lullabies Second ... | This was the only calender... |  5.0   |
|  Lamaze Peekaboo, I Love You  | I like how the book has a ... |  5.0   |
|  Lamaze Peekaboo, I Love You  | What a great book for babi... |  5.0   |
| SoftPlay Twinkle Twinkle E... |                               |  5.0   |
| SoftPlay Peek-A-Boo Where'... | We wanted to get another b... |  4.0   |
| SoftPlay Peek-A-Boo Where'... | My 3 month old son loves t... |  5.0   |
|   Our Baby Girl Memory Book   | Beautiful book, I love it ... |  5.0   |
| God Created Everything Nit... | This is well made a very p... |  5.0   |
| Cloth Diaper Pins Stainles... | A great safety pin that ha... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'it.': 1, 'a': 1, 'and': ... |    0    |   0   |     0     |    0    |  0   |
| {'and': 1, 'one': 3, 'seco... |    0    |   0   |     0     |    0    |  0   |
| {'family': 1, 'over': 1, '... |    0    |   1   |     0     |    0    |  0   |
| {'page.': 1, "couldn't": 1... |    0    |   1   |     0     |    0    |  0   |
|               {}              |    0    |   0   |     0     |    0    |  0   |
| {'and': 2, 'another': 1, '... |    0    |   0   |     0     |    0    |  0   |
| {'and': 7, 'old': 1, 'elmo... |    0    |   0   |     0     |    0    |  0   |
| {'pink': 1, 'beautiful': 2... |    0    |   1   |     0     |    0    |  1   |
| {'and': 1, 'is': 1, 'gift.... |    0    |   1   |     0     |    0    |  0   |
| {'and': 1, 'a': 3, 'great'... |    0    |   2   |     0     |    0    |  0   |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+
| horrible | bad | terrible | awful | wow | hate | sentiment |
+----------+-----+----------+-------+-----+------+-----------+
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
|    0     |  0  |    0     |   0   |  0  |  0   |     1     |
+----------+-----+----------+-------+-----+------+-----------+
[10 rows x 16 columns]

In [51]:
len(test_data)

33304

In [52]:
test_data['sentiment'].sum()

27976

In [54]:
27976/float(33304)

0.8400192169108815